<a href="https://colab.research.google.com/github/irisnoww/A-B-test/blob/main/A_B_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/baumanab/udacity_ABTesting#summary

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/12XHB60M1fE6k1E0_RwudkC-yjKBH3nSEMaFWiqn1CXM/edit#gid=0')

In [ ]:
#get data and transform to correct format
def data_input(tab):
  data = pd.DataFrame(wb.worksheet(tab).get_all_values())
  header = data.iloc[0]
  data = data[1:]
  data.columns = header
  data.fillna(0, inplace=True)
  data['Pageviews'] = data['Pageviews'].astype(int)
  data['Clicks'] = data['Clicks'].astype(int)
  # data['Enrollments'] = data['Enrollments'].replace(r'\s+', np.nan, regex=True)
  # data['Enrollments'] = data['Enrollments'].fillna(0)
  return data

In [ ]:
control_data = data_input('control')
exp_data = data_input('exp')

In [ ]:
control_data

In [ ]:
control_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 1 to 37
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         37 non-null     object
 1   Pageviews    37 non-null     int64 
 2   Clicks       37 non-null     int64 
 3   Enrollments  37 non-null     object
 4   Payments     37 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.6+ KB


In [ ]:
#convert first row to header
control_data = pd.DataFrame(wb.worksheet('control').get_all_values())
header = control_data.iloc[0]
control_data = control_data[1:]
control_data.columns = header

In [ ]:
exp_data = pd.DataFrame(wb.worksheet('exp').get_all_values())
header = exp_data.iloc[0]
exp_data = exp_data[1:]
exp_data.columns = header

In [ ]:
control_data['Pageviews']=control_data['Pageviews'].astype(int)

In [ ]:
exp_data['Pageviews']=exp_data['Pageviews'].astype(int)

### Experiment Analysis
**Sanity Checks**

Start by checking whether your invariant metrics are equivalent between the two groups. 


*   For a count (such as the first three metrics), you should calculate a confidence interval around the fraction of events you expect to be assigned to the control group, and the observed value should be the actual fraction that was assigned to the control group. This is the process that was shown in Lesson 5.

*   For any other type of metric, (such as the last four metrics), you should construct a confidence interval for a difference in proportions using a similar strategy as in Lesson 1, then check whether the difference between group values falls within that confidence level



```Unique cookies to view course overview page per day:	40000
Unique cookies to click "Start free trial" per day:	3200
Enrollments per day:	660
Click-through-probability on "Start free trial":	0.08
Probability of enrolling, given click:	0.20625
Probability of payment, given enroll:	0.53
Probability of payment, given click	0.1093125
```

In [ ]:
#number of cookies
#Sanity Checks for differences between counts
def sanity_ct(control_data, exp_data, metric,p):

  cnt = control_data[metric].sum()
  exp = exp_data[metric].sum()
  total = cnt + exp
  SE = np.sqrt((p*(1-p))/total)
  #zscore for 95% confidence interval
  ME = 1.96 * SE
  upper_bound = p + ME
  lower_bound = p - ME
  p_hat = cnt/total
  if p_hat <= upper_bound and p_hat >= lower_bound:
    print('pass')
  else:
    print('no')
  
  return upper_bound, lower_bound


In [ ]:
sanity_ct(control_data, exp_data, 'Pageviews',0.5)

pass


(0.5011796078509769, 0.49882039214902313)

In [ ]:
sanity_ct(control_data, exp_data,'Clicks',0.5)

pass


(0.5041155042762105, 0.49588449572378945)

In [ ]:
print(('the number of cookie upper bound {}, lower bound {}').format(p_upper,p_lower))

the number of cookie upper bound 0.5011796078509769, lower bound 0.49882039214902313


In [ ]:
def get_z_score(alpha):
  return norm.ppf(alpha)

In [ ]:
#Sanity Checks for differences between probabilities
#CTP
def sanity_prob(control_data, exp_data, metric_num, metric_deno,dmin):
  control_metric = control_data[metric_num].sum()/control_data[metric_deno].sum()
  exp_metric = exp_data[metric_num].sum()/exp_data[metric_deno].sum()
  pool_metric = (control_data[metric_num].sum() + exp_data[metric_num].sum())/(control_data[metric_deno].sum()+exp_data[metric_deno].sum())
  pool_sdv = np.sqrt((pool_metric*(1-pool_metric))*(1/(control_data['Pageviews'].sum()+1/exp_data['Pageviews'].sum())))
  ME = 1.96 *pool_sdv
  upper_bound = dmin + ME
  lower_bound = dmin - ME
  d_hat = control_metric - exp_metric

  if d_hat <= upper_bound and d_hat >= lower_bound:
    print('pass')
  else:
    print('no')
  return upper_bound, lower_bound


In [ ]:
sanity_prob(control_data, exp_data,'Clicks','Pageviews',0)

pass


(0.0009155973077741857, -0.0009155973077741857)

In [ ]:
dmin = control_ctr - exp_ctr

In [ ]:
\hat{p_{pool}}=\frac{x_{cont}+x_{exp}}{N_{cont}+N_{exp}}

### Experiment Analysis
**Evaluation Metrics Checks**

Significance definitions

A metric is statistically significant if the confidence interval **does not include 0** (that is, you can be confident there was a change), and it is practically significant if the confidence interval **does not include the practical significance boundary** (that is, you can be confident there is a change that matters to the business.)

Now, all that is left is to measure for each evaluation metric, the difference between the values from both groups. Then, we compute the confidence interval for that difference and test whether or not this confidence interval is both statistically and practically significant.


In [ ]:
control_data = data_input('control')[0:23]
exp_data = data_input('exp')[0:23]

In [ ]:
control_data['Enrollments'] = control_data['Enrollments'].astype(int)
exp_data['Enrollments'] = exp_data['Enrollments'].astype(int)

control_data['Payments'] = control_data['Payments'].astype(int)
exp_data['Payments'] = exp_data['Payments'].astype(int)

**Gross Conversion**

number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. (dmin= 0.01)

In [ ]:
import numpy as np


In [ ]:
def evaluation_check(control_data, exp_data, metric_num, metric_deno,dmin):
  control_metric = control_data[metric_num].sum()/control_data[metric_deno].sum()
  exp_metric = exp_data[metric_num].sum()/exp_data[metric_deno].sum()

  pool_metric = (control_data[metric_num].sum() + exp_data[metric_num].sum())/(control_data[metric_deno].sum()+exp_data[metric_deno].sum())
  pool_sdv = np.sqrt((pool_metric*(1-pool_metric))*(1/((control_data['Pageviews'].sum())+1/(exp_data['Pageviews'].sum()))))
  ME = 1.96 *pool_sdv
  d_hat =  exp_metric - control_metric
  upper_bound = d_hat + ME
  lower_bound = d_hat - ME
  if (0 not in np.arange(lower_bound,upper_bound)) and (dmin not in np.arange(lower_bound,upper_bound)):
    print('pass')
  else:
    print('no')
  return upper_bound, lower_bound

In [ ]:
evaluation_check(control_data, exp_data,'Enrollments','Clicks',0.01)

pass


(-0.018825926944296405, -0.022283822216426726)

Net conversion: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button. (dmin= 0.0075)

In [ ]:
control_data['Payments'] = control_data['Payments'].astype(int)
exp_data['Payments'] = exp_data['Payments'].astype(int)

In [ ]:
evaluation_check(control_data, exp_data,'Payments','Clicks',0.0075)

pass


(-0.003515562130606662, -0.0062318832184816726)

**Sign Tests**

For each of your evaluation metrics, do a sign test using the day-by-day data, and report the p-value of the sign test and whether the result is statistically significant. (These should be the answers from the "Sign Tests" quiz.)



In [ ]:
full_data = control_data.merge(exp_data, on='Date', how='left')

In [ ]:
full_data.head()

,Date,Pageviews_x,Clicks_x,Enrollments_x,Payments_x,Pageviews_y,Clicks_y,Enrollments_y,Payments_y
0,"Sat, Oct 11",7723,687,134,70,7716,686,105,34
1,"Sun, Oct 12",9102,779,147,70,9288,785,116,91
2,"Mon, Oct 13",10511,909,167,95,10480,884,145,79
3,"Tue, Oct 14",9871,836,156,105,9867,827,138,92
4,"Wed, Oct 15",10014,837,163,64,9793,832,140,94


In [ ]:
def critical_z(alpha=0.05, tail="two"):
    """
    Given significance level, compute critical value.
    """
    if tail == "two":
        p = 1 - alpha / 2
    else:
        p = 1 - alpha
        
    return norm.ppf(p)
  
def compute_power(h_0, h_1, se, alpha=0.05, tail="two"):
    """
    Compute power given the centers of sampling distributions
    under the null and alternative hypotheses, and shared standard error.
    """
    z = critical_z(alpha=alpha, tail=tail)
    
    lower = h_0 - z * se
    upper = h_0 + z * se
    
    lower_a = norm.cdf(lower, h_1, se)
    upper_a = 1 - norm.cdf(upper, h_1, se)
    
    if tail == "two":
        print("acceptance region [%.3f, %.3f]"%(lower, upper))
        return lower_a + upper_a
    
    elif tail == "left":
        print("acceptance region > %.3f"%(lower))
        return lower_a
    
    elif tail == "right":
        print("acceptance region < %.3f"%(upper))
        return upper_a